# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from usl_models.atmo_ml.model import AtmoModel, AtmoModelParams
from usl_models.atmo_ml import dataset
from google.cloud import storage

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
# Define bucket names and folder paths
data_bucket_name = "climateiq-study-area-feature-chunks"
label_bucket_name = "climateiq-study-area-label-chunks"
time_steps_per_day = 6
batch_size = 2

sim_dirs = [
    ('NYC_Heat_Test', [
        'NYC_summer_2000_01p',
        'NYC_summer_2010_25p',
        'NYC_summer_2015_50p',
        'NYC_summer_2017_75p',
        'NYC_summer_2018_99p'
    ]),
    ('PHX_Heat_Test', [
        'PHX_summer_2008_25p',
        'PHX_summer_2009_50p',
        'PHX_summer_2011_99p',
        'PHX_summer_2015_75p',
        'PHX_summer_2020_01p'
    ])
]

sim_names = []
for sim_dir, subdirs in sim_dirs:
    for subdir in subdirs:
        sim_names.append(sim_dir + '/' + subdir)

client = storage.Client(project="climateiq")


2024-12-11 22:46:47.208634: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 22:46:47.259060: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 22:46:47.259093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 22:46:47.260344: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 22:46:47.268340: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Create the training dataset using create_atmo_dataset with sim_names

train_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    timesteps_per_day=time_steps_per_day,
    hash_range=(0.0, 0.5),
).batch(batch_size=batch_size)

val_ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    timesteps_per_day=time_steps_per_day,
    hash_range=(0.5, 1.0),
).batch(batch_size=batch_size)

2024-12-11 22:46:50.791744: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38364 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


In [ ]:
for i, _ in enumerate(train_ds):
    print(i)

In [24]:
# Initialize the Atmo Model
model_params = AtmoModelParams()
model = AtmoModel(model_params)
# model._model.summary()

In [ ]:
# Train the model
model.fit(train_ds, val_ds, epochs=1)

Split shape: (None, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (None, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (None, 50, 50, 8, 256)
Split shape: (None, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (None, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (None, 50, 50, 8, 256)


2024-12-11 22:45:25.409753: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inatmo_conv_lstm_5/conv_lstm/conv_lstm2d_5/while/body/_1/atmo_conv_lstm_5/conv_lstm/conv_lstm2d_5/while/dropout_7/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Output shape: TensorShape([None, 8, 50, 50, 256])
load_day('2000-05-24', 'NYC_Heat_Test/NYC_summer_2000_01p')
spatiotemporal_data: (6, 200, 200, 12)
load_day('2000-05-28', 'NYC_Heat_Test/NYC_summer_2000_01p')
spatiotemporal_data: (6, 200, 200, 12)
load_day('2000-05-29', 'NYC_Heat_Test/NYC_summer_2000_01p')
      1/Unknown - 9s 9s/step - loss: 17994.4375 - mean_absolute_error: 73.8874 - root_mean_squared_error: 134.1433Output shape: TensorShape([None, 8, 50, 50, 256])
spatiotemporal_data: (6, 200, 200, 12)
load_day('2000-05-30', 'NYC_Heat_Test/NYC_summer_2000_01p')
spatiotemporal_data: (6, 200, 200, 12)
load_day('2000-06-03', 'NYC_Heat_Test/NYC_summer_2000_01p')
      2/Unknown - 12s 3s/step - loss: 17710.3223 - mean_absolute_error: 73.2867 - root_mean_squared_error: 133.0801Output shape: TensorShape([None, 8, 50, 50, 256])
spatiotemporal_data: (6, 200, 200, 12)
load_day('2000-06-04', 'NYC_Heat_Test/NYC_summer_2000_01p')
spatiotemporal_data: (6, 200, 200, 12)
load_day('2000-06-06', 'NYC

In [ ]:
model._model.summary()

Model: "atmo_conv_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  488       
                                                                 
 spatial_cnn (Sequential)    (None, 50, 50, 128)       252992    
                                                                 
 spatiotemporal_cnn (Sequen  (None, None, 50, 50, 64   30480     
 tial)                       )                                   
                                                                 
 conv_lstm (Sequential)      (None, None, 50, 50, 51   45877248  
                             2)                                  
                                                                 
 t2_output_cnn (Sequential)  (None, None, 200, 200,    69729     
                             1)                                  
                                                    

In [ ]:
# Test calling the model on some training data
inputs, labels = next(iter(train_dataset))
prediction = model.call(inputs)
print("Prediction shape:", prediction.shape)

Split shape: (2, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (2, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (2, 50, 50, 8, 256)
Output shape: TensorShape([2, 8, 50, 50, 256])
Prediction shape: (2, 8, 200, 200, 5)
